In [1]:
import lightgbm as lgb
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import missingno as msno
import datetime
import seaborn as sns
import gc
import re
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import RandomForestClassifier
from sklearn.multioutput import MultiOutputRegressor
from sklearn.model_selection import train_test_split
from sklearn import preprocessing
# from pandarallel import pandarallel
# pandarallel.initialize()
plt.rcParams['font.sans-serif']=['SimHei']
plt.rcParams['axes.unicode_minus'] = False
#显示所有列
pd.set_option('display.max_columns', 100)
#显示所有行
pd.set_option('display.max_rows', 100)
#设置value的显示长度为100，默认为50
pd.set_option('max_colwidth',70)

In [2]:
df_train=pd.read_pickle("../processed_data/final_process_train.pkl")
df_test=pd.read_pickle("../processed_data/final_process_test.pkl")

In [3]:
del df_train["truetime"]
del df_test["truetime"]

In [4]:
df_train

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,os,osv,package,timestamp,version,fea_hash,location,fea1_hash,cus_type,day,hour,minute
sid,,,,,,,,,,,,,,,,,,,,,,
1438873,316361,81,1,1098.0,480.0,912.0,1,1.0,21,6,1,114,19.0,1.559893e+12,13.0,0,1,1503,16,7,7,32
1185582,135939,31,0,721.0,480.0,1233.0,1,7.0,4,6,1,112,3.0,1.559994e+12,8.0,0,2,1852,37,8,11,40
1555716,399254,22,0,760.0,140.0,360.0,1,7.0,111,0,1,113,8.0,1.559837e+12,2.0,10594,3,406,21,6,15,58
1093419,68983,49,1,2214.0,480.0,1080.0,0,7.0,26,2,1,113,25.0,1.560042e+12,9.0,0,4,846,24,9,0,59
1400089,288999,62,1,2280.0,480.0,1080.0,1,7.0,13,2,1,109,6.0,1.559867e+12,9.0,0,5,985,15,7,0,28
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546078,392477,53,1,1920.0,3.0,1080.0,1,7.0,29,6,0,78,26.0,1.559834e+12,12.0,0,80,85,4,6,15,14
1480612,346134,48,0,1424.0,2.0,720.0,0,7.0,6,2,1,113,5.0,1.559814e+12,7.0,0,24,1765,12,6,9,40
1698442,499635,12,1,1280.0,320.0,720.0,0,7.0,11,6,1,66,10.0,1.559676e+12,13.0,0,26,872,27,4,19,14


## 有理特征工程

In [5]:
df_train["hw_ratio"]=df_train.dev_height/df_train.dev_width
df_test["hw_ratio"]=df_test.dev_height/df_test.dev_width
df_train["hw_matrix"]=df_train.dev_height*df_train.dev_width
df_test["hw_matrix"]=df_test.dev_height*df_test.dev_width
df_train["inch"]=(df_train.dev_height**2+df_train.dev_width**2)**0.5/df_train.dev_ppi
df_test["inch"]=(df_test.dev_height**2+df_test.dev_width**2)**0.5/df_test.dev_ppi

## 统计特征工程
单一特征：count/500000

In [6]:
def statcount(df_,col,f,type1="train"):
    name=col+"_count"
    print("{},{} start!".format(type1,name))
    df=df_.copy()
    if type=="train":num_all=500000
    else:num_all=150000
    order_label=df.groupby([col])[f].count()/num_all
    df[name] = df[col].map(order_label)
    return df

In [7]:
for col in ["apptype","carrier","dev_ppi","dev_width","lan","media_id","ntt","osv","package","version","fea_hash","location","fea1_hash","cus_type","day"]:
    df_train=statcount(df_train,col,"android_id",type1="train")
    df_test=statcount(df_test,col,"android_id",type1="test")

train,apptype_count start!
test,apptype_count start!
train,carrier_count start!
test,carrier_count start!
train,dev_ppi_count start!
test,dev_ppi_count start!
train,dev_width_count start!
test,dev_width_count start!
train,lan_count start!
test,lan_count start!
train,media_id_count start!
test,media_id_count start!
train,ntt_count start!
test,ntt_count start!
train,osv_count start!
test,osv_count start!
train,package_count start!
test,package_count start!
train,version_count start!
test,version_count start!
train,fea_hash_count start!
test,fea_hash_count start!
train,location_count start!
test,location_count start!
train,fea1_hash_count start!
test,fea1_hash_count start!
train,cus_type_count start!
test,cus_type_count start!
train,day_count start!
test,day_count start!


In [8]:
df_train

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,os,osv,package,timestamp,version,fea_hash,location,fea1_hash,cus_type,day,hour,minute,hw_ratio,hw_matrix,inch,apptype_count,carrier_count,dev_ppi_count,dev_width_count,lan_count,media_id_count,ntt_count,osv_count,package_count,version_count,fea_hash_count,location_count,fea1_hash_count,cus_type_count,day_count
sid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1438873,316361,81,1,1098.0,480.0,912.0,1,1.0,21,6,1,114,19.0,1.559893e+12,13.0,0,1,1503,16,7,7,32,1.203947,1001376.0,2.973660,0.154747,2.396060,1.335540,0.086527,0.166113,0.148780,0.776987,0.522780,0.149180,0.519420,2.557173,0.077647,0.166460,0.043040,0.494340
1185582,135939,31,0,721.0,480.0,1233.0,1,7.0,4,6,1,112,3.0,1.559994e+12,8.0,0,2,1852,37,8,11,40,0.584753,888993.0,2.975690,0.181033,0.271013,1.335540,0.045567,2.888920,0.058860,0.776987,0.128673,0.104700,0.330473,2.557173,0.036847,0.013360,0.043200,0.502033
1555716,399254,22,0,760.0,140.0,360.0,1,7.0,111,0,1,113,8.0,1.559837e+12,2.0,10594,3,406,21,6,15,58,2.111111,273600.0,6.006799,0.010027,0.271013,0.000747,0.514180,2.888920,0.001893,0.137447,1.063080,0.041773,0.242340,0.000033,0.010240,0.001893,0.043060,0.504867
1093419,68983,49,1,2214.0,480.0,1080.0,0,7.0,26,2,1,113,25.0,1.560042e+12,9.0,0,4,846,24,9,0,59,2.050000,2391120.0,5.132023,0.111473,2.396060,1.335540,1.399593,2.888920,0.013873,2.123980,1.063080,0.105513,0.586367,2.557173,0.009720,0.000800,0.042753,0.411213
1400089,288999,62,1,2280.0,480.0,1080.0,1,7.0,13,2,1,109,6.0,1.559867e+12,9.0,0,5,985,15,7,0,28,2.111111,2462400.0,5.255949,0.267887,2.396060,1.335540,1.399593,2.888920,0.267833,2.123980,0.108080,0.322000,0.586367,2.557173,0.037320,0.044533,0.043227,0.494340
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546078,392477,53,1,1920.0,3.0,1080.0,1,7.0,29,6,0,78,26.0,1.559834e+12,12.0,0,80,85,4,6,15,14,1.777778,2073600.0,734.302390,0.028187,2.396060,0.568593,1.399593,2.888920,0.024127,0.776987,0.081867,0.024127,0.186540,2.557173,0.019233,0.000667,0.042487,0.504867
1480612,346134,48,0,1424.0,2.0,720.0,0,7.0,6,2,1,113,5.0,1.559814e+12,7.0,0,24,1765,12,6,9,40,1.977778,1025280.0,797.837076,0.674987,0.271013,0.450213,0.422113,2.888920,0.626520,2.123980,1.063080,0.626520,0.932880,2.557173,0.055233,0.067600,0.042860,0.504867
1698442,499635,12,1,1280.0,320.0,720.0,0,7.0,11,6,1,66,10.0,1.559676e+12,13.0,0,26,872,27,4,19,14,1.777778,921600.0,4.589390,0.262840,2.396060,0.241647,0.422113,2.888920,0.246700,0.776987,0.215367,0.318520,0.519420,2.557173,0.025193,0.001267,0.044067,0.502120


聚合特征：count/500000，unique


In [9]:
def day_group_count(df_,col,f):
    name=col+"_"+ f +"_group_count"
    print("{} start!".format(name))
    df=df_.copy()
    order_label=df.groupby([col])[f].value_counts()
    order_label.index =['{}_{}'.format(i, j) for i, j in order_label.index]
    df[name] =['{}_{}'.format(i, j) for i, j in zip(df[col],df[f])]
    df[name] =df[name].map(order_label)
    # df[name] =df[name]
    df["xxx"]=df[col].map(df[col].value_counts())
    df[name] /= df["xxx"]
    del df["xxx"]
    return df

In [10]:
for col in ["apptype","carrier","dev_ppi","dev_width","lan","media_id","ntt","osv","package","version","fea_hash","location","fea1_hash","cus_type"]:
    df_train=day_group_count(df_train,"day",col)
    df_test=day_group_count(df_test,"day",col)


day_apptype_group_count start!
day_apptype_group_count start!
day_carrier_group_count start!
day_carrier_group_count start!
day_dev_ppi_group_count start!
day_dev_ppi_group_count start!
day_dev_width_group_count start!
day_dev_width_group_count start!
day_lan_group_count start!
day_lan_group_count start!
day_media_id_group_count start!
day_media_id_group_count start!
day_ntt_group_count start!
day_ntt_group_count start!
day_osv_group_count start!
day_osv_group_count start!
day_package_group_count start!
day_package_group_count start!
day_version_group_count start!
day_version_group_count start!
day_fea_hash_group_count start!
day_fea_hash_group_count start!
day_location_group_count start!
day_location_group_count start!
day_fea1_hash_group_count start!
day_fea1_hash_group_count start!
day_cus_type_group_count start!
day_cus_type_group_count start!


In [11]:
df_train

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,os,osv,package,timestamp,version,fea_hash,location,fea1_hash,cus_type,day,hour,minute,hw_ratio,hw_matrix,inch,apptype_count,carrier_count,dev_ppi_count,dev_width_count,lan_count,media_id_count,ntt_count,osv_count,package_count,version_count,fea_hash_count,location_count,fea1_hash_count,cus_type_count,day_count,day_apptype_group_count,day_carrier_group_count,day_dev_ppi_group_count,day_dev_width_group_count,day_lan_group_count,day_media_id_group_count,day_ntt_group_count,day_osv_group_count,day_package_group_count,day_version_group_count,day_fea_hash_group_count,day_location_group_count,day_fea1_hash_group_count,day_cus_type_group_count
sid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1438873,316361,81,1,1098.0,480.0,912.0,1,1.0,21,6,1,114,19.0,1.559893e+12,13.0,0,1,1503,16,7,7,32,1.203947,1001376.0,2.973660,0.154747,2.396060,1.335540,0.086527,0.166113,0.148780,0.776987,0.522780,0.149180,0.519420,2.557173,0.077647,0.166460,0.043040,0.494340,0.058421,0.723942,0.408734,0.031921,0.062238,0.056614,0.220874,0.157867,0.056763,0.161684,0.780718,0.022360,0.062696,0.012704
1185582,135939,31,0,721.0,480.0,1233.0,1,7.0,4,6,1,112,3.0,1.559994e+12,8.0,0,2,1852,37,8,11,40,0.584753,888993.0,2.975690,0.181033,0.271013,1.335540,0.045567,2.888920,0.058860,0.776987,0.128673,0.104700,0.330473,2.557173,0.036847,0.013360,0.043200,0.502033,0.048363,0.084496,0.400584,0.012602,0.862346,0.015935,0.213107,0.035018,0.029971,0.086488,0.785858,0.010796,0.003731,0.013186
1555716,399254,22,0,760.0,140.0,360.0,1,7.0,111,0,1,113,8.0,1.559837e+12,2.0,10594,3,406,21,6,15,58,2.111111,273600.0,6.006799,0.010027,0.271013,0.000747,0.514180,2.888920,0.001893,0.137447,1.063080,0.041773,0.242340,0.000033,0.010240,0.001893,0.043060,0.504867,0.002483,0.087271,0.000119,0.156675,0.860108,0.000317,0.038241,0.316823,0.011884,0.072125,0.000026,0.003446,0.000317,0.013509
1093419,68983,49,1,2214.0,480.0,1080.0,0,7.0,26,2,1,113,25.0,1.560042e+12,9.0,0,4,846,24,9,0,59,2.050000,2391120.0,5.132023,0.111473,2.396060,1.335540,1.399593,2.888920,0.013873,2.123980,1.063080,0.105513,0.586367,2.557173,0.009720,0.000800,0.042753,0.411213,0.027934,0.719432,0.381311,0.410055,0.870529,0.002902,0.653546,0.320288,0.026945,0.175221,0.784783,0.003145,0.000259,0.013067
1400089,288999,62,1,2280.0,480.0,1080.0,1,7.0,13,2,1,109,6.0,1.559867e+12,9.0,0,5,985,15,7,0,28,2.111111,2462400.0,5.255949,0.267887,2.396060,1.335540,1.399593,2.888920,0.267833,2.123980,0.108080,0.322000,0.586367,2.557173,0.037320,0.044533,0.043227,0.494340,0.077491,0.723942,0.408734,0.401357,0.852949,0.077491,0.659708,0.029561,0.092204,0.190247,0.780718,0.010964,0.012475,0.013513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546078,392477,53,1,1920.0,3.0,1080.0,1,7.0,29,6,0,78,26.0,1.559834e+12,12.0,0,80,85,4,6,15,14,1.777778,2073600.0,734.302390,0.028187,2.396060,0.568593,1.399593,2.888920,0.024127,0.776987,0.081867,0.024127,0.186540,2.557173,0.019233,0.000667,0.042487,0.504867,0.004278,0.714538,0.168876,0.428668,0.860108,0.003645,0.242308,0.023782,0.003645,0.051697,0.760967,0.005414,0.000145,0.012624
1480612,346134,48,0,1424.0,2.0,720.0,0,7.0,6,2,1,113,5.0,1.559814e+12,7.0,0,24,1765,12,6,9,40,1.977778,1025280.0,797.837076,0.674987,0.271013,0.450213,0.422113,2.888920,0.626520,2.123980,1.063080,0.626520,0.932880,2.557173,0.055233,0.067600,0.042860,0.504867,0.202549,0.087271,0.134293,0.129275,0.860108,0.189647,0.631454,0.316823,0.189647,0.281566,0.760967,0.016955,0.020586,0.012901
1698442,499635,12,1,1280.0,320.0,720.0,0,7.0,11,6,1,66,10.0,1.559676e+12,13.0,0,26,872,27,4,19,14,1.777778,921600.0,4.589390,0.262840,2.396060,0.241647,0.422113,2.888920,0.246700,0.776987,0.215367,0.318520,0.519420,2.557173,0.025193,0.001267,0.044067,0.502120,0.080698,0.705276,0.075812,0.119985,0.

In [12]:
def hour_group_count(df_,col,f):
    df=df_.copy()
    name_col="_".join(col)
    df[name_col]=['{}_{}'.format(i, j) for i, j in zip(df[col[0]],df[col[1]])]
    name="hour_"+ f +"_group_count"
    print("{} start!".format(name))
    order_label=df.groupby([name_col])[f].value_counts()
    order_label.index =['{}_{}'.format(i, j) for i, j in order_label.index]
    df[name] =['{}_{}'.format(i, j) for i, j in zip(df[name_col],df[f])]
    df[name] =df[name].map(order_label)
    df["xxx"]=df[name_col].map(df[name_col].value_counts())
    df[name] /= df["xxx"]
    del df["xxx"]
    del df[name_col]
    return df

In [13]:
for col in ["apptype","carrier","dev_ppi","dev_width","lan","media_id","ntt","osv","package","version","fea_hash","location","fea1_hash","cus_type"]:
    df_train=hour_group_count(df_train,["day","hour"],col)
    df_test=hour_group_count(df_test,["day","hour"],col)

hour_apptype_group_count start!
hour_apptype_group_count start!
hour_carrier_group_count start!
hour_carrier_group_count start!
hour_dev_ppi_group_count start!
hour_dev_ppi_group_count start!
hour_dev_width_group_count start!
hour_dev_width_group_count start!
hour_lan_group_count start!
hour_lan_group_count start!
hour_media_id_group_count start!
hour_media_id_group_count start!
hour_ntt_group_count start!
hour_ntt_group_count start!
hour_osv_group_count start!
hour_osv_group_count start!
hour_package_group_count start!
hour_package_group_count start!
hour_version_group_count start!
hour_version_group_count start!
hour_fea_hash_group_count start!
hour_fea_hash_group_count start!
hour_location_group_count start!
hour_location_group_count start!
hour_fea1_hash_group_count start!
hour_fea1_hash_group_count start!
hour_cus_type_group_count start!
hour_cus_type_group_count start!


In [14]:
df_train

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,os,osv,package,timestamp,version,fea_hash,location,fea1_hash,cus_type,day,hour,minute,hw_ratio,hw_matrix,inch,apptype_count,carrier_count,dev_ppi_count,dev_width_count,lan_count,media_id_count,ntt_count,osv_count,package_count,version_count,fea_hash_count,location_count,fea1_hash_count,cus_type_count,day_count,day_apptype_group_count,day_carrier_group_count,day_dev_ppi_group_count,day_dev_width_group_count,day_lan_group_count,day_media_id_group_count,day_ntt_group_count,day_osv_group_count,day_package_group_count,day_version_group_count,day_fea_hash_group_count,day_location_group_count,day_fea1_hash_group_count,day_cus_type_group_count,hour_apptype_group_count,hour_carrier_group_count,hour_dev_ppi_group_count,hour_dev_width_group_count,hour_lan_group_count,hour_media_id_group_count,hour_ntt_group_count,hour_osv_group_count,hour_package_group_count,hour_version_group_count,hour_fea_hash_group_count,hour_location_group_count,hour_fea1_hash_group_count,hour_cus_type_group_count
sid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1438873,316361,81,1,1098.0,480.0,912.0,1,1.0,21,6,1,114,19.0,1.559893e+12,13.0,0,1,1503,16,7,7,32,1.203947,1001376.0,2.973660,0.154747,2.396060,1.335540,0.086527,0.166113,0.148780,0.776987,0.522780,0.149180,0.519420,2.557173,0.077647,0.166460,0.043040,0.494340,0.058421,0.723942,0.408734,0.031921,0.062238,0.056614,0.220874,0.157867,0.056763,0.161684,0.780718,0.022360,0.062696,0.012704,0.067503,0.726725,0.405521,0.039398,0.072020,0.065747,0.231368,0.149561,0.065747,0.167629,0.776662,0.019322,0.072773,0.011041
1185582,135939,31,0,721.0,480.0,1233.0,1,7.0,4,6,1,112,3.0,1.559994e+12,8.0,0,2,1852,37,8,11,40,0.584753,888993.0,2.975690,0.181033,0.271013,1.335540,0.045567,2.888920,0.058860,0.776987,0.128673,0.104700,0.330473,2.557173,0.036847,0.013360,0.043200,0.502033,0.048363,0.084496,0.400584,0.012602,0.862346,0.015935,0.213107,0.035018,0.029971,0.086488,0.785858,0.010796,0.003731,0.013186,0.049222,0.084417,0.405509,0.015557,0.838052,0.016832,0.234889,0.040041,0.029329,0.082887,0.776588,0.011732,0.004336,0.013262
1555716,399254,22,0,760.0,140.0,360.0,1,7.0,111,0,1,113,8.0,1.559837e+12,2.0,10594,3,406,21,6,15,58,2.111111,273600.0,6.006799,0.010027,0.271013,0.000747,0.514180,2.888920,0.001893,0.137447,1.063080,0.041773,0.242340,0.000033,0.010240,0.001893,0.043060,0.504867,0.002483,0.087271,0.000119,0.156675,0.860108,0.000317,0.038241,0.316823,0.011884,0.072125,0.000026,0.003446,0.000317,0.013509,0.001318,0.089902,0.000264,0.104930,0.836805,0.000264,0.031374,0.309518,0.007118,0.074347,0.000264,0.003691,0.000264,0.015819
1093419,68983,49,1,2214.0,480.0,1080.0,0,7.0,26,2,1,113,25.0,1.560042e+12,9.0,0,4,846,24,9,0,59,2.050000,2391120.0,5.132023,0.111473,2.396060,1.335540,1.399593,2.888920,0.013873,2.123980,1.063080,0.105513,0.586367,2.557173,0.009720,0.000800,0.042753,0.411213,0.027934,0.719432,0.381311,0.410055,0.870529,0.002902,0.653546,0.320288,0.026945,0.175221,0.784783,0.003145,0.000259,0.013067,0.033597,0.738193,0.352773,0.400951,0.890650,0.003803,0.661807,0.325198,0.032330,0.163867,0.793661,0.004120,0.000317,0.010777
1400089,288999,62,1,2280.0,480.0,1080.0,1,7.0,13,2,1,109,6.0,1.559867e+12,9.0,0,5,985,15,7,0,28,2.111111,2462400.0,5.255949,0.267887,2.396060,1.335540,1.399593,2.888920,0.267833,2.123980,0.108080,0.322000,0.586367,2.557173,0.037320,0.044533,0.043227,0.494340,0.077491,0.723942,0.408734,0.401357,0.852949,0.077491,0.659708,0.029561,0.092204,0.190247,0.780718,0.010964,0.012475,0.013513,0.070280,0.728333,0.376300,0.398046,0.886858,0.070280,0.667192,0.033722,0.088875,0.163883,0.792310,0.012606,0.011346,0.015443
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1546078,392477,53,1,192

## 组合特征工程

In [15]:
def combine_fea(df_,col1,col2):
    df=df_.copy() 
    name="{}_{}_combine".format(col1,col2)
    print("{} start!".format(name))
    df[name]=['{}_{}'.format(i, j) for i, j in zip(df[col1],df[col2])]
    return df

In [16]:
## 用户和设备
user_col=["fea1_hash","fea_hash","cus_type"]
dev_col=["ntt","carrier","osv","lan","dev_height","dev_width","dev_ppi"]
media_col=["apptype","media_id","version","package"]

for col1,col2 in zip(user_col,dev_col):
    df_train=combine_fea(df_train,col1,col2)
    df_test=combine_fea(df_test,col1,col2)

for col1,col2 in zip(user_col,media_col):
    df_train=combine_fea(df_train,col1,col2)
    df_test=combine_fea(df_test,col1,col2)

for col1,col2 in zip(media_col,dev_col):
    df_train=combine_fea(df_train,col1,col2)
    df_test=combine_fea(df_test,col1,col2)

fea1_hash_ntt_combine start!
fea1_hash_ntt_combine start!
fea_hash_carrier_combine start!
fea_hash_carrier_combine start!
cus_type_osv_combine start!
cus_type_osv_combine start!
fea1_hash_apptype_combine start!
fea1_hash_apptype_combine start!
fea_hash_media_id_combine start!
fea_hash_media_id_combine start!
cus_type_version_combine start!
cus_type_version_combine start!
apptype_ntt_combine start!
apptype_ntt_combine start!
media_id_carrier_combine start!
media_id_carrier_combine start!
version_osv_combine start!
version_osv_combine start!
package_lan_combine start!
package_lan_combine start!


## Save

In [17]:
df_train

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,label,lan,media_id,ntt,os,osv,package,timestamp,version,fea_hash,location,fea1_hash,cus_type,day,hour,minute,hw_ratio,hw_matrix,inch,apptype_count,carrier_count,dev_ppi_count,dev_width_count,lan_count,media_id_count,ntt_count,osv_count,package_count,version_count,fea_hash_count,location_count,fea1_hash_count,cus_type_count,day_count,day_apptype_group_count,day_carrier_group_count,day_dev_ppi_group_count,day_dev_width_group_count,day_lan_group_count,day_media_id_group_count,day_ntt_group_count,day_osv_group_count,day_package_group_count,day_version_group_count,day_fea_hash_group_count,day_location_group_count,day_fea1_hash_group_count,day_cus_type_group_count,hour_apptype_group_count,hour_carrier_group_count,hour_dev_ppi_group_count,hour_dev_width_group_count,hour_lan_group_count,hour_media_id_group_count,hour_ntt_group_count,hour_osv_group_count,hour_package_group_count,hour_version_group_count,hour_fea_hash_group_count,hour_location_group_count,hour_fea1_hash_group_count,hour_cus_type_group_count,fea1_hash_ntt_combine,fea_hash_carrier_combine,cus_type_osv_combine,fea1_hash_apptype_combine,fea_hash_media_id_combine,cus_type_version_combine,apptype_ntt_combine,media_id_carrier_combine,version_osv_combine,package_lan_combine
sid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1438873,316361,81,1,1098.0,480.0,912.0,1,1.0,21,6,1,114,19.0,1.559893e+12,13.0,0,1,1503,16,7,7,32,1.203947,1001376.0,2.973660,0.154747,2.396060,1.335540,0.086527,0.166113,0.148780,0.776987,0.522780,0.149180,0.519420,2.557173,0.077647,0.166460,0.043040,0.494340,0.058421,0.723942,0.408734,0.031921,0.062238,0.056614,0.220874,0.157867,0.056763,0.161684,0.780718,0.022360,0.062696,0.012704,0.067503,0.726725,0.405521,0.039398,0.072020,0.065747,0.231368,0.149561,0.065747,0.167629,0.776662,0.019322,0.072773,0.011041,1503_6,0_1,16_114,1503_81,0_21,16_13.0,81_6,21_1,13.0_114,19.0_1.0
1185582,135939,31,0,721.0,480.0,1233.0,1,7.0,4,6,1,112,3.0,1.559994e+12,8.0,0,2,1852,37,8,11,40,0.584753,888993.0,2.975690,0.181033,0.271013,1.335540,0.045567,2.888920,0.058860,0.776987,0.128673,0.104700,0.330473,2.557173,0.036847,0.013360,0.043200,0.502033,0.048363,0.084496,0.400584,0.012602,0.862346,0.015935,0.213107,0.035018,0.029971,0.086488,0.785858,0.010796,0.003731,0.013186,0.049222,0.084417,0.405509,0.015557,0.838052,0.016832,0.234889,0.040041,0.029329,0.082887,0.776588,0.011732,0.004336,0.013262,1852_6,0_0,37_112,1852_31,0_4,37_8.0,31_6,4_0,8.0_112,3.0_7.0
1555716,399254,22,0,760.0,140.0,360.0,1,7.0,111,0,1,113,8.0,1.559837e+12,2.0,10594,3,406,21,6,15,58,2.111111,273600.0,6.006799,0.010027,0.271013,0.000747,0.514180,2.888920,0.001893,0.137447,1.063080,0.041773,0.242340,0.000033,0.010240,0.001893,0.043060,0.504867,0.002483,0.087271,0.000119,0.156675,0.860108,0.000317,0.038241,0.316823,0.011884,0.072125,0.000026,0.003446,0.000317,0.013509,0.001318,0.089902,0.000264,0.104930,0.836805,0.000264,0.031374,0.309518,0.007118,0.074347,0.000264,0.003691,0.000264,0.015819,406_0,10594_0,21_113,406_22,10594_111,21_2.0,22_0,111_0,2.0_113,8.0_7.0
1093419,68983,49,1,2214.0,480.0,1080.0,0,7.0,26,2,1,113,25.0,1.560042e+12,9.0,0,4,846,24,9,0,59,2.050000,2391120.0,5.132023,0.111473,2.396060,1.335540,1.399593,2.888920,0.013873,2.123980,1.063080,0.105513,0.586367,2.557173,0.009720,0.000800,0.042753,0.411213,0.027934,0.719432,0.381311,0.410055,0.870529,0.002902,0.653546,0.320288,0.026945,0.175221,0.784783,0.003145,0.000259,0.013067,0.033597,0.738193,0.352773,0.400951,0.890650,0.003803,0.661807,0.325198,0.032330,0.163867,0.793661,0.004120,0.000317,0.010777,846_2,0_1,24_113,846_49,0_26,24_9.0,49_2,26_1,9.0_113,25.0_7.0
1400089,288999,62,1,2280.0,480.0,1080.0,1,7.0,13,2,1,109,6.0,1.559867e+12,9.0,0,5,985,15,7,0,28,2.111111,2462400.0,5.255949,0.267887,2.396060,1.335540,1.399593,2.888920,0.267833,2.123980,0.108080,0.322000,0.586367,2.557173,0.037320,0.044533,0.043227,0.494340,0.077491,0.723942,0.408734,0.401357,0.852949,0.

In [18]:
df_test

,android_id,apptype,carrier,dev_height,dev_ppi,dev_width,lan,media_id,ntt,os,osv,package,timestamp,version,fea_hash,location,fea1_hash,cus_type,label,day,hour,minute,hw_ratio,hw_matrix,inch,apptype_count,carrier_count,dev_ppi_count,dev_width_count,lan_count,media_id_count,ntt_count,osv_count,package_count,version_count,fea_hash_count,location_count,fea1_hash_count,cus_type_count,day_count,day_apptype_group_count,day_carrier_group_count,day_dev_ppi_group_count,day_dev_width_group_count,day_lan_group_count,day_media_id_group_count,day_ntt_group_count,day_osv_group_count,day_package_group_count,day_version_group_count,day_fea_hash_group_count,day_location_group_count,day_fea1_hash_group_count,day_cus_type_group_count,hour_apptype_group_count,hour_carrier_group_count,hour_dev_ppi_group_count,hour_dev_width_group_count,hour_lan_group_count,hour_media_id_group_count,hour_ntt_group_count,hour_osv_group_count,hour_package_group_count,hour_version_group_count,hour_fea_hash_group_count,hour_location_group_count,hour_fea1_hash_group_count,hour_cus_type_group_count,fea1_hash_ntt_combine,fea_hash_carrier_combine,cus_type_osv_combine,fea1_hash_apptype_combine,fea_hash_media_id_combine,cus_type_version_combine,apptype_ntt_combine,media_id_carrier_combine,version_osv_combine,package_lan_combine
sid,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
1440682,317625,78,1,2196.0,2.0,1080.0,1.0,125,2,0,113,183.0,1.559872e+12,12.0,0,58,2555,19,None,7,1,43,2.033333,2371680.0,1223.602877,0.000260,0.717940,0.136547,0.420513,0.049267,0.000260,0.63692,0.316960,0.000260,0.056093,0.837080,0.011940,0.000267,0.012793,0.147920,0.000225,0.724941,0.141833,0.405084,0.062601,0.000225,0.663827,0.305886,0.000225,0.049216,0.846133,0.010772,0.000225,0.012394,0.002028,0.712982,0.148073,0.383367,0.046653,0.002028,0.628803,0.310345,0.002028,0.050710,0.835700,0.012170,0.002028,0.019270,2555_2,0_1,19_113,2555_78,0_125,19_12.0,78_2,125_1,12.0_113,183.0_1.0
1606824,435108,41,3,2280.0,3.0,1080.0,7.0,137,6,0,113,210.0,1.559739e+12,7.0,0,24,72,34,None,5,12,53,2.111111,2462400.0,840.951842,0.002247,0.064713,0.171140,0.420513,0.867087,0.000440,0.23250,0.316960,0.000440,0.280113,0.837080,0.017013,0.020047,0.012860,0.147547,0.003434,0.064522,0.146937,0.415597,0.870188,0.000542,0.245165,0.323694,0.000542,0.299747,0.827173,0.017847,0.017125,0.012019,0.004345,0.068791,0.169442,0.394642,0.884142,0.002172,0.219406,0.341057,0.002172,0.262129,0.852281,0.020275,0.053584,0.015930,72_6,0_3,34_113,72_41,0_137,34_7.0,41_6,137_3,7.0_113,210.0_7.0
1774642,0,68,1,1280.0,2.0,810.0,7.0,8,2,1,55,0.0,1.559614e+12,6.0,0,31,2796,6,None,4,2,8,1.580247,1036800.0,757.380354,0.095160,0.717940,0.136547,0.002593,0.867087,0.074260,0.63692,0.038993,0.040213,0.110193,0.837080,0.015013,0.002307,0.268260,0.152000,0.080088,0.700395,0.137149,0.002105,0.880351,0.060877,0.611491,0.035219,0.036009,0.101316,0.827412,0.016535,0.004649,0.271535,0.071222,0.682387,0.154957,0.002887,0.872955,0.056785,0.558229,0.030799,0.037536,0.101059,0.808470,0.015399,0.007700,0.277190,2796_2,0_1,6_55,2796_68,0_8,6_6.0,68_2,8_1,6.0_55,0.0_7.0
1742535,451504,12,1,1344.0,320.0,720.0,7.0,11,2,1,77,10.0,1.559668e+12,13.0,11452,66,2188,29,None,4,17,3,1.866667,967680.0,4.764714,0.080627,0.717940,0.072533,0.125460,0.867087,0.075687,0.63692,0.055313,0.097647,0.156860,0.000013,0.001007,0.006953,0.012827,0.152000,0.081491,0.700395,0.074737,0.118026,0.880351,0.078158,0.611491,0.056667,0.099167,0.149781,0.000088,0.001272,0.006798,0.013070,0.081448,0.642534,0.090498,0.144796,0.904977,0.079186,0.665158,0.067873,0.097285,0.138009,0.002262,0.002262,0.002262,0.027149,2188_2,11452_1,29_77,2188_12,11452_11,29_13.0,12_2,11_1,13.0_77,10.0_7.0
1689686,0,48,1,665.0,1.0,320.0,7.0,6,5,0,113,5.0,1.559694e+12,7.0,0,149,1677,6,None,5,0,16,2.078125,212800.0,737.987127,0.202513,0.717940,0.173293,0.007767,0.867087,0.187920,0.07014,0.316960,0.187920,0.280113,0.837080,0.003533,0.020013,0.268260,0.147547,0.221896,0.720721,0.190222,0.008088,0.8

In [19]:
df_train.to_pickle("../processed_data/final_feature_train.pkl")
df_test.to_pickle("../processed_data/final_feature_test.pkl")